### Import required Python libraries and set plotting parameters 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import time
import os
import os.path
import zipfile
import pandas as pd
from scipy.optimize import curve_fit, least_squares, brute, brent
import gmsh
import math
import pyvista as pv
import re
import pickle

import ipywidgets
from ipywidgets import Play, IntProgress, link, HBox, AppLayout, Dropdown, VBox, Dropdown, jslink, GridspecLayout, IntSlider
from ipygany import Scene, PolyMesh, TetraMesh, IsoColor, Threshold, IsoSurface, colormaps, ColorBar, Warp
from IPython.display import clear_output, display

from matplotlib.colors import ListedColormap
pv.set_plot_theme("document")

plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = "DejaVu Serif"
plt.rcParams['font.size'] = 20

from pyvirtualdisplay import Display
virtual_display = Display(backend="xvfb", visible=False, size=(800, 600))
virtual_display.start()
    
user_name=!whoami # get user name
user_name=user_name[0]
um_view = "/mofem_install/jupyter/%s/um_view" % user_name

### Define utility functions including black-box launch of MoFEM


In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")
    def __setattr__(self, key, value):
        self[key] = value

def replace_template_sdf(params):
    regex = r"\{(.*?)\}"
    with open(params.template_sdf_file) as infile, open(params.sdf_file, 'w') as outfile:
        for line in infile:
            matches = re.finditer(regex, line, re.DOTALL)
            for match in matches:
                for name in match.groups():
                    src = "{" + name + "}"
                    target = str(params[name])
                    line = line.replace(src, target)
            outfile.write(line)

def get_young_modulus(K, G):
    E = 9. * K * G /(3. * K + G)
    return E

def get_poisson_ratio(K, G):
    nu = (3. * K - 2. * G) / 2. / (3. * K + G)
    return nu

def get_bulk_modulus(E, nu):
    K = E / 3. / (1. - 2. * nu)
    return K

def get_shear_modulus(E, nu):
    G = E / 2. / (1. + nu)
    return G

def parse_log_file(params):
    time, indent, force = [], [], []
    with open(params.log_file, "r") as log_file:
        for line in log_file:
            line = line.strip()
            if "Contact force" in line:
                line = line.split()
                time.append(float(line[6]))
                force.append(float(line[8]))
            if "Uy" in line:
                line = line.split()
                indent.append(abs(float(line[7])))
    return np.array(time), np.array(indent), np.array(force)

def partition_mesh(params):
    params.part_file = os.path.splitext(params.mesh_file)[0] + "_" + str(params.nproc) + "p.h5m"
    
    !{um_view}/bin/mofem_part \
    -my_file {params.mesh_file} \
    -my_nparts {params.nproc} \
    -output_file {params.part_file} \
    -dim 3 -adj_dim 1

def run_deformation_analysis(params):    
    !rm -rf out_mi*
    
    mi_param_2 = 0
    mi_param_3 = 0
    mi_param_4 = 0
    
    if params.material_model == "LinearElasticity":
        mi_block = "LinearElasticity"
        mi_param_0 = params.young_modulus
        mi_param_1 = params.poisson_ratio
    elif params.material_model == "SaintVenantKirchhoffElasticity":
        mi_block = "SaintVenantKirchhoffElasticity"
        mi_param_0 = params.young_modulus
        mi_param_1 = params.poisson_ratio
    elif params.material_model == "NeoHookeanHyperElasticity":
        mi_block = "SignoriniHyperElasticity"
        mi_param_0 = get_bulk_modulus(params.young_modulus, params.poisson_ratio)
        mi_param_1 = 0.5 * get_shear_modulus(params.young_modulus, params.poisson_ratio)
    elif params.material_model == "StandardLinearSolid":
        mi_block = "StandardLinearSolid"
        mi_param_0 = get_bulk_modulus(params.young_modulus_0, params.poisson_ratio_0)
        mi_param_1 = get_shear_modulus(params.young_modulus_0, params.poisson_ratio_0)
        mi_param_2 = get_bulk_modulus(params.young_modulus_1, params.poisson_ratio_1)
        mi_param_3 = get_shear_modulus(params.young_modulus_1, params.poisson_ratio_1)
        mi_param_4 = params.relax_time_1
    else:
        print("Unknown material model: " + params.material_model)
        return
    
    partition_mesh(params)
    
    ts_type = "beuler"
    if not params.quasi_static:
        ts_type = "alpha2"
        
    !export OMPI_MCA_btl_vader_single_copy_mechanism=none && \
    nice -n 10 mpirun --oversubscribe --allow-run-as-root \
    -np {params.nproc} {um_view}/tutorials/adv-1/contact_3d \
    -file_name {params.part_file} \
    -order {params.order} \
    -ts_dt {params.time_step} \
    -ts_max_time {params.final_time} \
    -rho {params.density} \
    -alpha_damping {params.damping} \
    -is_quasi_static {int(params.quasi_static)} \
    -ts_type {ts_type} \
    -mi_lib_path_{params.mfront_block_id} {um_view}/mfront_interface/libBehaviour.so \
    -mi_block_{params.mfront_block_id} {mi_block} \
    -mi_param_{params.mfront_block_id}_0 {mi_param_0} \
    -mi_param_{params.mfront_block_id}_1 {mi_param_1} \
    -mi_param_{params.mfront_block_id}_2 {mi_param_2} \
    -mi_param_{params.mfront_block_id}_3 {mi_param_3} \
    -mi_param_{params.mfront_block_id}_4 {mi_param_4} \
    -mi_save_volume {int(params.save_volume)} \
    -mi_save_gauss 0 \
    -save_every {params.save_step} \
    -no_contact \
    2>&1 | tee {params.log_file}
    
    if params.save_volume:
        !convert.py -np {params.nproc} out*
    
    return 

def run_eigen_analysis(params):  
    !rm -rf out_eig*
    partition_mesh(params)
        
    !export OMPI_MCA_btl_vader_single_copy_mechanism=none && \
    nice -n 10 mpirun --oversubscribe --allow-run-as-root \
    -np {params.nproc} {um_view}/tutorials/vec-1/eigen_elastic_3d \
    -file_name {params.part_file} \
    -rho {params.density} \
    -young_modulus {params.young_modulus} \
    -poisson_ratio {params.poisson_ratio} \
    -eps_tol 1e-3 \
    -eps_nev {params.mode_num} \
    2>&1 | tee {params.log_file}
    
    if params.save_volume:
        !convert.py -np {params.nproc} out*
    
    return 
            
def show_mesh(params):
    !mbconvert {params.mesh_file} {params.vtk_file}

    mesh = pv.read(params.vtk_file)
    mesh = mesh.shrink(0.95)

    p = pv.Plotter(notebook=True)
    p.add_mesh(mesh, color="lightgrey", show_edges=True, smooth_shading=False, cmap="turbo")

    p.show(jupyter_backend='ipygany')
    
    return

def play_animation(params, eigen_analysis=False):
    if not eigen_analysis:
        file_prefix = "out_mi_"
        warp_field = "DISPLACEMENT"
        step_num = params['final_time'] / params['time_step']
        step = params['save_step']
        description='Time [s]:'
        mult = params['time_step']
        factor=1
    else:
        file_prefix = "out_eig_"
        warp_field = "U"
        step_num = params.mode_num - 1
        step = 1
        description = 'Eigenmode:'
        mult = 1
        factor=100
        
    mesh = PolyMesh.from_vtk("{}0.vtk".format(file_prefix))
    
    field_drop_names = []
    field_tuples = []
    for data in mesh.data:
        if data.name != "GLOBAL_ID":
            for i, comp in enumerate(data.components):
                field_tuples.append((data.name, comp.name))
                drop_name = data.name
                if data.dim > 1:
                    drop_name += "_" + str(i)
                field_drop_names.append((drop_name, len(field_drop_names)))
                
    fields_drop = Dropdown(
        options=field_drop_names,
        description='Field:',
        value = 0
    )   
    
    warped_mesh = Warp(mesh, input=warp_field, factor=factor)
    iso = IsoColor(warped_mesh)
    iso.input = field_tuples[0]
    
    iso.colormap = colormaps.Turbo
    colorbar = ColorBar(iso)
    scene = Scene([iso])
    
    colormap = Dropdown(
        options=colormaps,
        description='colormap:'
    )
    jslink((iso, 'colormap'), (colormap, 'index'))
    

    
    play = Play(description='Step:', min=0, max=step_num, step=step, value=0)
    slider = IntSlider(min=0, max=step_num, step=step, value=0)
    jslink((play, 'value'), (slider, 'value'))
    stepper = HBox((play, slider))

    time_text = ipywidgets.FloatText(description=description, disabled=True)

    grid = GridspecLayout(1, 2)
    grid[0, 0] = VBox((stepper, time_text))
    grid[0, 1] = VBox((HBox((fields_drop, colormap)), colorbar))

    app = AppLayout(header=grid, center=scene)
    
    def update_range(field_name, comp_name):
        comp = int(comp_name[-1]) - 1
        for data in mesh.data:
            if data.name == field_name:
                array = data.components[comp].array
                iso.range = [min(array), max(array)]   
    
    def value_changed(change):
        time_text.value = change.new * mult
        mesh.reload('{}{}.vtk'.format(file_prefix, str(change.new)), reload_vertices=False)
        update_range(*field_tuples[fields_drop.value])
        
    def field_changed(change):
        iso.input = field_tuples[int(change.new)]
        update_range(*field_tuples[int(change.new)])
        clear_output()
        display(app)

    update_range(*field_tuples[0])
    
    slider.observe(value_changed, 'value')
    fields_drop.observe(field_changed, 'value')
    
    return app

### Set simulation parameters

In [ ]:
params = AttrDict()

params.mesh_file = "cantilever.cub"
params.vtk_file = "mesh_2d.vtk"

params.load_history_file = "load.txt"
params.log_file = "log_indent"

params.mfront_block_id = 1 

params.nproc = 8 # number of processors/cores used
params.order = 2 #order of approximation functions

params.save_volume = True 
params.save_step = 1

### Show input mesh

In [ ]:
show_mesh(params)

### Run deformation analysis

In [ ]:
params.material_model = "LinearElasticity" 
params.young_modulus = 170e3 #[MPa = nN / um^2]
params.poisson_ratio = 0.4
params.density = 2.283e-12 #[nN / um ^4 * s]
params.damping = 0

params.quasi_static = False

params.time_step = 0.01 #[s]
params.final_time = 1 #[s]

run_deformation_analysis(params)

In [ ]:
play_animation(params)

### Run eigen mode analysis

In [ ]:
params.young_modulus = 170e3 #[MPa = nN / um^2]
params.poisson_ratio = 0.4
params.density = 2.329e-15 #[kg/um^3]

params.mode_num = 5

run_eigen_analysis(params)

### Post-process

In [ ]:
# Get frequencies 
frequency_log='frequency_log'
!grep "frequency" {params.log_file} > {frequency_log} 
frequencies_data=pd.read_csv(frequency_log,sep='\s+',header=None)
frequencies=frequencies_data[14].to_numpy()
print(frequencies)

for f in zip(range(1,params.mode_num+1),frequencies):
    print('Mode ',f[0],' frequency ',f[1],' Hz')
    
plt.barh(range(1,params.mode_num+1), frequencies, align='center')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Vibration mode')

In [ ]:
play_animation(params, eigen_analysis=True)